In [61]:
import os
import sys

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers

# image dataset parameters
val_size = 0.2  
batch_size = 32
img_height = 128 
img_width = 128 

# create training dataset
train_data = tf.keras.preprocessing.image_dataset_from_directory(
  "Data",
  validation_split = val_size,
  subset = "training",
  seed = 123,
  color_mode = 'grayscale',
  image_size = (img_height, img_width),
  batch_size = batch_size)

# create validation dataset
val_data = tf.keras.preprocessing.image_dataset_from_directory(
  "Data",
  validation_split = val_size,
  subset = "validation",
  seed = 123,
  color_mode = "grayscale",
  image_size = (img_height, img_width),
  batch_size = batch_size)

# optimisation
AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
val_data = val_data.cache().prefetch(buffer_size=AUTOTUNE)

# neural network parameters
hidden_units = 128
dropout_rate = 0.2  
learning_rate = 1e-5  

# creating neural network (MLP model)
model = tf.keras.Sequential([
  tf.keras.Input(shape=(img_height, img_width, 1)),
  tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
  tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset=-1),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(hidden_units, activation='relu'),
  tf.keras.layers.Dropout(rate=dropout_rate),
  tf.keras.layers.Dense(hidden_units, activation='relu'),
  tf.keras.layers.Dense(5)
], name="MLP")

model.compile(
  optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate),
  loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics = ["accuracy"]
)

model.summary()

history = model.fit(train_data,validation_data=val_data,epochs=20)


Found 6517 files belonging to 5 classes.
Using 5214 files for training.
Found 6517 files belonging to 5 classes.
Using 1303 files for validation.
Model: "MLP"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
random_flip_11 (RandomFlip)  (None, 128, 128, 1)       0         
_________________________________________________________________
rescaling_11 (Rescaling)     (None, 128, 128, 1)       0         
_________________________________________________________________
flatten_11 (Flatten)         (None, 16384)             0         
_________________________________________________________________
dense_33 (Dense)             (None, 128)               2097280   
_________________________________________________________________
dropout_11 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_34 (Dense)             (None, 128)         

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [64]:
# attempt to make prediction based on inputted image

input_data = tf.keras.preprocessing.image_dataset_from_directory(
  "test",
  validation_split=val_size,
  subset="validation",
  seed=123,
  color_mode='grayscale',
  image_size=(128, 128),
  batch_size=1)

model.predict(
    input_data,
    batch_size=None,
    verbose=2,
    steps=None,
    callbacks=None,
    use_multiprocessing=False
)

Found 1 files belonging to 1 classes.
Using 0 files for validation.
1/1 - 0s


array([[ 0.86872876,  0.21257624,  0.32524022,  0.7548242 , -1.3887922 ]],
      dtype=float32)

In [67]:
# attempt 2 to create inputted image

from PIL import Image

class_names = ["Action", "Animation", "Comedy", "Drama", "Horror"]
img = tf.keras.utils.load_img(
    "test/bee/bee-movie-2007-us-poster.jpg", 
    target_size=(128, 128)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch
predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)




This image most likely belongs to Drama with a 44.11 percent confidence.


In [53]:
# save and load model

model.save('cinemaizer.h5')
tf.keras.models.load_model('cinemaizer.h5').summary()

Model: "MLP"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
random_flip_10 (RandomFlip)  (None, 128, 128, 1)       0         
_________________________________________________________________
rescaling_10 (Rescaling)     (None, 128, 128, 1)       0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 16384)             0         
_________________________________________________________________
dense_30 (Dense)             (None, 128)               2097280   
_________________________________________________________________
dropout_10 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_31 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_32 (Dense)             (None, 5)                 645     